# Dealing with the Data



In [ ]:
import os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

TheDataGuy's blog posts are in markdown in git repo, that why we copied the docs here for initial version

- 14 posts currently

TODO:

[ ]  - Develop a pipeline to ingest data as new posts are published


In [ ]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
text_loader = DirectoryLoader(path, glob="*.md", show_progress=True,recursive=True)


In [7]:
raw_docs = text_loader.load()

100%|██████████| 14/14 [00:00<00:00, 41.74it/s]


In [15]:
len(raw_docs)

14

In [ ]:
# add url in metadata by replace "data/" with "https://thedataguy.pro/" and remove "index.md" from metadata.source
for doc in raw_docs:
    doc.metadata["url"] = doc.metadata["source"].replace("data/", "https://thedataguy.pro/blog/").replace("index.md", "")






In [129]:
# build list of doc source and text length
doc_info = []
for doc in raw_docs:
    doc_info.append(
        {
            "url": doc.metadata["url"],
            "source": doc.metadata["source"],
            "text_length": len(doc.page_content),
        }
    )



In [130]:
max(len(doc.page_content) for doc in raw_docs)

13468

The longest blog post contains 13,468 characters. Ideally, we would like to retrieve full post content based on the query. 

> That's why chunking based on the blog post might be the best way.

In [131]:
# display as markdown table
import pandas as pd
doc_info_df = pd.DataFrame(doc_info)

from IPython.display import display
display(doc_info_df)
 

,url,source,text_length
0,https://thedataguy.pro/introduction-to-ragas/,data/introduction-to-ragas/index.md,6071
1,https://thedataguy.pro/generating-test-data-wi...,data/generating-test-data-with-ragas/index.md,13468
2,https://thedataguy.pro/advanced-metrics-and-cu...,data/advanced-metrics-and-customization-with-r...,10455
3,https://thedataguy.pro/building-research-agent/,data/building-research-agent/index.md,6877
4,https://thedataguy.pro/rss-feed-announcement/,data/rss-feed-announcement/index.md,1900
5,https://thedataguy.pro/metric-driven-development/,data/metric-driven-development/index.md,11879
6,https://thedataguy.pro/basic-evaluation-workfl...,data/basic-evaluation-workflow-with-ragas/inde...,9164
7,https://thedataguy.pro/langchain-experience-cs...,data/langchain-experience-csharp-perspective/i...,3070
8,https://thedataguy.pro/evaluating-ai-agents-wi...,data/evaluating-ai-agents-with-ragas/index.md,8907
9,https://thedataguy.pro/integrations-and-observ...,data/integrations-and-observability-with-ragas...,8221


In [ ]:
# calculate the number of tokens in the documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)
docs = text_splitter.split_documents(raw_docs)
len(docs)
# check the first document
print(docs[0].page_content[:1000])
# check the metadata
print(docs[0].metadata)


title: "Part 1: Introduction to Ragas: The Essential Evaluation Framework for LLM Applications" date: 2025-04-26T18:00:00-06:00 layout: blog description: "Explore the essential evaluation framework for LLM applications with Ragas. Learn how to assess performance, ensure accuracy, and improve reliability in Retrieval-Augmented Generation systems." categories: ["AI", "RAG", "Evaluation","Ragas"] coverImage: "https://images.unsplash.com/photo-1593642634367-d91a135587b5?q=80&w=1770&auto=format&fit=crop&ixlib=rb-4.0.3" readingTime: 7 published: true

As Large Language Models (LLMs) become fundamental components of modern applications, effectively evaluating their performance becomes increasingly critical. Whether you're building a question-answering system, a document retrieval tool, or a conversational agent, you need reliable metrics to assess how well your application performs. This is where Ragas steps in.

What is Ragas?
{'source': 'data/introduction-to-ragas/index.md'}


In [9]:
len(docs)

139

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
huggingface_embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-l")

In [133]:
from langchain_community.vectorstores import Qdrant

In [ ]:
import os
from pathlib import Path

# Create directory if it doesn't exist
storage_path = Path("./db")
os.makedirs(storage_path, exist_ok=True)

qdrant_vectorstore = Qdrant.from_documents(
    documents=docs,
    embedding=huggingface_embeddings,
    path=str(storage_path / "vectorstore"),
    collection_name="thedataguy_documents",
)

In [135]:
# Create directory if it doesn't exist
storage_path = Path("./db")
os.makedirs(storage_path, exist_ok=True)

qdrant_vectorstore = Qdrant.from_documents(
    documents=raw_docs,
    embedding=huggingface_embeddings,
    path=str(storage_path / "vectorstore_v3"),
    collection_name="thedataguy_documents",
)

In [145]:
# display content in markdown
from IPython.display import Markdown, display
def display_markdown(content):
    display(Markdown(content))

In [136]:
retriever = qdrant_vectorstore.as_retriever()

In [137]:
query = "Who is TheDataGuy?"
retriever.invoke(query,k=2)

[Document(metadata={'source': 'data/advanced-metrics-and-customization-with-ragas/index.md', 'url': 'https://thedataguy.pro/advanced-metrics-and-customization-with-ragas/', '_id': 'dea41eaa40a54b3ca558fda681d69ec8', '_collection_name': 'thedataguy_documents'}, page_content='title: "Part 5: Advanced Metrics and Customization with Ragas" date: 2025-04-28T05:00:00-06:00 layout: blog description: "Explore advanced metrics and customization techniques in Ragas for evaluating LLM applications, including creating custom metrics, domain-specific evaluation, composite scoring, and best practices for building a comprehensive evaluation ecosystem." categories: ["AI", "RAG", "Evaluation", "Ragas","Data"] coverImage: "https://plus.unsplash.com/premium_photo-1661368994107-43200954c524?q=80&w=2070&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D" readingTime: 9 published: true\n\nIn our previous post, we explored how to generate comprehensive test datasets

In [138]:
from langchain.prompts import ChatPromptTemplate

rag_prompt_template = """\
You are a helpful assistant that answers questions based on the context provided. 
Generate a concise answer to the question in markdown format and include a list of relevant links to the context.
Use links from context to help user to navigate to to find more information.
You have access to the following information:

Context:
{context}

Question:
{question}

If context is unrelated to question, say "I don't know".
"""

rag_prompt = ChatPromptTemplate.from_template(rag_prompt_template)

In [140]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

In [142]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | llm, "context": itemgetter("context")}
)

In [143]:
query = "Who is TheDataGuy?"
response = retrieval_augmented_qa_chain.invoke({"question" : query})

In [144]:
response["response"].content

"I don't know."

In [146]:
query = "What is RAGAS?"
response = retrieval_augmented_qa_chain.invoke({"question" : query})

In [147]:
response

{'response': AIMessage(content='Ragas is an open-source evaluation framework specifically designed for Large Language Model (LLM) applications, particularly focusing on Retrieval-Augmented Generation (RAG) systems. It provides specialized metrics to assess the performance of LLM applications, addressing challenges such as information retrieval accuracy, response consistency, and user query relevance. Ragas helps ensure that LLM applications are reliable and effective by offering tools for quality assurance, performance tracking, and continuous improvement.\n\nFor more information, you can explore the following links:\n- [Part 1: Introduction to Ragas](https://thedataguy.pro/introduction-to-ragas/)\n- [Part 2: Basic Evaluation Workflow with Ragas](https://thedataguy.pro/basic-evaluation-workflow-with-ragas/)\n- [Part 3: Evaluating RAG Systems with Ragas](https://thedataguy.pro/evaluating-rag-systems-with-ragas/)\n- [Part 5: Advanced Metrics and Customization with Ragas](https://thedatag

In [148]:
display_markdown(response["response"].content)

Ragas is an open-source evaluation framework specifically designed for Large Language Model (LLM) applications, particularly focusing on Retrieval-Augmented Generation (RAG) systems. It provides specialized metrics to assess the performance of LLM applications, addressing challenges such as information retrieval accuracy, response consistency, and user query relevance. Ragas helps ensure that LLM applications are reliable and effective by offering tools for quality assurance, performance tracking, and continuous improvement.

For more information, you can explore the following links:
- [Part 1: Introduction to Ragas](https://thedataguy.pro/introduction-to-ragas/)
- [Part 2: Basic Evaluation Workflow with Ragas](https://thedataguy.pro/basic-evaluation-workflow-with-ragas/)
- [Part 3: Evaluating RAG Systems with Ragas](https://thedataguy.pro/evaluating-rag-systems-with-ragas/)
- [Part 5: Advanced Metrics and Customization with Ragas](https://thedataguy.pro/advanced-metrics-and-customization-with-ragas/)

In [149]:
query = "How to build research agents?"
response = retrieval_augmented_qa_chain.invoke({"question" : query})

In [150]:
response

{'response': AIMessage(content="To build a research agent, you can follow these key steps:\n\n1. **Define the Purpose**: Identify the specific research needs, such as searching across multiple sources, analyzing documents, and providing structured reports.\n\n2. **Integrate Information Sources**:\n   - **Web Search**: Use APIs like Tavily and DuckDuckGo for real-time information.\n   - **Academic Research**: Connect to databases like arXiv for scholarly articles.\n   - **RSS Feeds**: Implement an RSS feed reader to aggregate content from relevant publications.\n\n3. **Document Analysis**: Incorporate a document analysis engine that uses techniques like Retrieval Augmented Generation (RAG) to process and analyze uploaded documents.\n\n4. **Workflow Architecture**: Utilize a structured framework like LangGraph for orchestrating the research process, maintaining context, and ensuring transparent reasoning.\n\n5. **Technology Stack**: Leverage tools such as LangChain, Chainlit, Qdrant, and

In [ ]:
display_markdown(response["response"].content)

To build a research agent, you can follow these key steps:

1. **Define the Purpose**: Identify the specific research needs, such as searching across multiple sources, analyzing documents, and providing structured reports.

2. **Integrate Information Sources**:
   - **Web Search**: Use APIs like Tavily and DuckDuckGo for real-time information.
   - **Academic Research**: Connect to databases like arXiv for scholarly articles.
   - **RSS Feeds**: Implement an RSS feed reader to aggregate content from relevant publications.

3. **Document Analysis**: Incorporate a document analysis engine that uses techniques like Retrieval Augmented Generation (RAG) to process and analyze uploaded documents.

4. **Workflow Architecture**: Utilize a structured framework like LangGraph for orchestrating the research process, maintaining context, and ensuring transparent reasoning.

5. **Technology Stack**: Leverage tools such as LangChain, Chainlit, Qdrant, and OpenAI's GPT-4o for building the agent.

6. **Testing and Iteration**: Continuously test the agent's performance and refine its capabilities based on user feedback and research outcomes.

For more detailed guidance, you can explore the following resources:
- [Building a Research Agent with RSS Feed Support](https://thedataguy.pro/building-research-agent/)
- [Advanced Metrics and Customization with Ragas](https://thedataguy.pro/advanced-metrics-and-customization-with-ragas/)
- [Evaluating RAG Systems with Ragas](https://thedataguy.pro/evaluating-rag-systems-with-ragas/)
- [Evaluating AI Agents with Ragas](https://thedataguy.pro/evaluating-ai-agents-with-ragas/)